In [1]:
# Quantum-Classical Hybrid Model using BERT and Quantum Kernel SVM for GoEmotions Dataset

# Step 1: Environment Setup (Google Colab)
!pip uninstall -y jax jaxlib
!pip install jax==0.4.28 jaxlib==0.4.28 --quiet
!pip install pennylane seaborn tensorflow-datasets scikit-learn==1.6.1 transformers --upgrade --quiet

Found existing installation: jax 0.5.2
Uninstalling jax-0.5.2:
  Successfully uninstalled jax-0.5.2
Found existing installation: jaxlib 0.5.1
Uninstalling jaxlib-0.5.1:
  Successfully uninstalled jaxlib-0.5.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 MB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
orbax-checkpoint 0.11.16 requires jax>=0.5.0, but you have jax 0.4.28 which is incompatible.
flax 0.10.6 requires jax>=0.5.1, but you have jax 0.4.28 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10

In [2]:
# Step 2: Import Required Libraries
import pennylane as qml
from pennylane import numpy as np
import numpy as onp
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from transformers import BertTokenizer, BertModel
import tensorflow_datasets as tfds
import torch
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Step 3: Load GoEmotions Dataset
print("Loading GoEmotions dataset...")
dataset, info = tfds.load('goemotions', with_info=True)
train_dataset = dataset['train']

# Define emotion labels in the correct order
emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
    'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
    'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral',
    'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise'
]

texts, labels = [], []
for example in tfds.as_numpy(train_dataset):
    texts.append(example['comment_text'].decode('utf-8'))
    for idx, label in enumerate(emotion_labels):
        if example[label]:
            labels.append(idx)
            break
    else:
        labels.append(20)

n_classes = 28

Loading GoEmotions dataset...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/goemotions/incomplete.B8ZGYZ_0.1.0/goemotions-validation.tfrecord*...:   0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/goemotions/incomplete.B8ZGYZ_0.1.0/goemotions-test.tfrecord*...:   0%|    …

Dataset goemotions downloaded and prepared to /root/tensorflow_datasets/goemotions/0.1.0. Subsequent calls will reuse this data.


In [4]:
# Step 4: BERT Embedding Extraction
print("Extracting BERT embeddings...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to compute BERT [CLS] embeddings
def bert_embed(sentences):
    with torch.no_grad():
        inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=128)
        outputs = model(**inputs)
        return outputs.last_hidden_state[:, 0, :].numpy()

batch_size = 64
bert_embeddings = []
for i in range(0, len(texts), batch_size):
    bert_embeddings.append(bert_embed(texts[i:i+batch_size]))
X = onp.vstack(bert_embeddings)
y = onp.array(labels)

Extracting BERT embeddings...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
# Step 5: Dimensionality Reduction
print("Reducing BERT embedding dimensions using PCA...")
pca = PCA(n_components=16)
X_reduced = pca.fit_transform(X)

# Limit dataset to 5000 samples for computational feasibility
X = X[:5000]
y = y[:5000]

# Step 5: Dimensionality Reduction
print("Reducing BERT embedding dimensions using PCA...")
pca = PCA(n_components=16)
X_reduced = pca.fit_transform(X)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

Reducing BERT embedding dimensions using PCA...
Reducing BERT embedding dimensions using PCA...


In [7]:
# Step 6: Quantum Kernel Setup
n_qubits = 16
dev = qml.device('default.qubit', wires=n_qubits)

def feature_map(x):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    qml.CNOT(wires=[n_qubits - 1, 0])

@qml.qnode(dev)
def kernel_circuit(x1, x2):
    feature_map(x1)
    qml.adjoint(feature_map)(x2)
    return qml.probs(wires=range(n_qubits))

# Quantum kernel function
from tqdm import tqdm

def quantum_kernel(X1, X2):
    kernel = np.zeros((len(X1), len(X2)))
    for i in tqdm(range(len(X1)), desc='Computing quantum kernel rows'):
        for j in range(len(X2)):
            kernel[i, j] = np.abs(kernel_circuit(X1[i], X2[j])[0])
    return kernel

In [ ]:
# Step 7: Compute Quantum Kernel Matrices
print("Computing quantum kernel matrices...")
K_train = quantum_kernel(X_train, X_train)
K_test = quantum_kernel(X_test, X_train)

Computing quantum kernel matrices...


Computing quantum kernel rows:   0%|          | 19/4000 [1:20:20<284:11:54, 257.00s/it]

In [ ]:
# Step 8: Train SVM with Precomputed Quantum Kernel
print("Training Quantum Kernel SVM...")
svm = SVC(kernel='precomputed')
svm.fit(K_train, y_train)

In [ ]:
# Step 9: Evaluation
y_pred = svm.predict(K_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")

print("Detailed Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Step 10: Confusion Matrix Plot
plt.figure(figsize=(12,10))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()